<h4>Preprocessing Map Locations by Data Visualization</h4>

<h5>Import Dependencies</h5>

In [1]:
!pip install w3lib
!pip install selenium
!pip install geocoder
!pip install geopy
!pip install geopy
!pip install folium

import numpy as np  # library to handle data in a vectorized manner
import pandas as pd  # library for data analsysis
import requests
import csv
import pprint as pp
import time
import re
import urllib.parse

from w3lib.url import url_query_parameter
from selenium import webdriver as wd
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from geopy import distance

# from selenium.common.exceptions import IndexError

from bs4 import BeautifulSoup as bs  # Beautiful Soup v4

# Only use when full dataframe needs to be viewed
# pd.set_option("display.max_rows", None, "display.max_columns", None)

import geocoder  # import geocoder

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium  # plotting library

print('Libraries imported.')

Libraries imported.


<h5>Extract Contents of Excel File into DataFrame and View</h5>

In [2]:
akl_cbd_biz = pd.read_excel("address_to_lat_lon_Cleaned.xlsx")

k = akl_cbd_biz.shape[0]

akl_cbd_biz.head()

,Name,Main Category,Sub Category 1,Sub Category 2,Address Line 1,Address Line 2,Precinct,Post Code,Locality,Phone,Web URL,Web URL COVID Info,Latitude,Longitude
0,100 Fix,Retail,Electronics and technology,NaN,NaN,19 Shortland Street,\n\nFederal & Elliott \n\n,1010.0,Auckland,,NaN,No URL,-36.846757,174.767031
1,109 Doctors,Services,Health and wellbeing,Doctors,Level 9,109 Queen Street,\n\nQueen Street \n\n,NaN,Auckland,(09)3660109,NaN,No URL,-36.854640,174.763338
2,16 Tun,Hospitality,Bars and clubs,Restaurants,NaN,10/26 Jellicoe Street,\n\nViaduct & Waterfront \n\n,NaN,Auckland,09 368 7712,www.16tun.com,URL Parse Error,-36.841196,174.757551
3,1Above Ltd,Services,Health and wellbeing,NaN,Old Sofrana House Level 1,18-Customs Street East,\n\nViaduct & Waterfront \n\n,NaN,Auckland,09 379 5708,www.fly1above.com,No Obvious COVID Info,-36.844742,174.767495
4,2 Degrees,Retail,Electronics and technology,NaN,Civic Centre,8 Wellesley Street West,\n\nFederal & Elliott \n\n,NaN,Auckland,09 366 1584,www.2degreesmobile.co.nz,No Obvious COVID Info,-36.850853,174.763871


<h5>Define Auckland Map</h5>

In [3]:
map_center_lat = -36.848461
map_center_lng = 174.763336
map_center = (map_center_lat, map_center_lng)
map_zoom_start = 13.5

<h5>Display Empty Auckland Map</h5>

In [8]:
akl_map = folium.Map(
    location=[map_center_lat, map_center_lng], zoom_start=map_zoom_start)
akl_map

<h5>Superimpose and Display Location Markers on the Map</h5>

In [9]:
# instantiate a feature group for the locations in the dataframe
locations = folium.map.FeatureGroup()

# loop through the locations and add each to the locations feature group
for lat, lng, in zip(akl_cbd_biz.Latitude, akl_cbd_biz.Longitude):
    locations.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5,  # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
akl_map.add_child(locations)

# Save and display populated Auckland map
akl_map

<h5>Apply Geofencing to Calculate Distance of Locations From City Center and Determine Outlier Locations</h5>

In [12]:
rad = float(1.5000)  # in kilometers
rad_m = rad * 1000   # in meters

geofence_l = []
distance_l = []

c = 0
for c in range(0, k):
    # extract latitude and longitude from DataFrame
    lat = akl_cbd_biz['Latitude'][c]
    lng = akl_cbd_biz['Longitude'][c]
    test_point = (lat, lng)

    dis = distance.great_circle(map_center, test_point).km
    dis = format(dis, '.4f')
    dis = float(dis)
    #print("Distance: {}".format(dis)) 
    if dis <= rad:
        geo = 'Inside'
        #print("{} point is inside the {} km radius from {} coordinate".format(
        #    test_point, rad, map_center))
    else:
        geo = 'Outside'
        #print("{} point is outside the {} km radius from {} coordinate".format(
        #    test_point, rad, map_center))

    geofence_l.append(geo)
    distance_l.append(dis)

# Append Geofence and Distance Data to DataFrame

akl_cbd_biz['Geofence'] = geofence_l
akl_cbd_biz['Distance from City Center'] = distance_l
akl_cbd_biz.head()

,Name,Main Category,Sub Category 1,Sub Category 2,Address Line 1,Address Line 2,Precinct,Post Code,Locality,Phone,Web URL,Web URL COVID Info,Latitude,Longitude,Geofence,Distance from City Center
0,100 Fix,Retail,Electronics and technology,NaN,NaN,19 Shortland Street,\n\nFederal & Elliott \n\n,1010.0,Auckland,,NaN,No URL,-36.846757,174.767031,Inside,0.3795
1,109 Doctors,Services,Health and wellbeing,Doctors,Level 9,109 Queen Street,\n\nQueen Street \n\n,NaN,Auckland,(09)3660109,NaN,No URL,-36.854640,174.763338,Inside,0.6870
2,16 Tun,Hospitality,Bars and clubs,Restaurants,NaN,10/26 Jellicoe Street,\n\nViaduct & Waterfront \n\n,NaN,Auckland,09 368 7712,www.16tun.com,URL Parse Error,-36.841196,174.757551,Inside,0.9579
3,1Above Ltd,Services,Health and wellbeing,NaN,Old Sofrana House Level 1,18-Customs Street East,\n\nViaduct & Waterfront \n\n,NaN,Auckland,09 379 5708,www.fly1above.com,No Obvious COVID Info,-36.844742,174.767495,Inside,0.5550
4,2 Degrees,Retail,Electronics and technology,NaN,Civic Centre,8 Wellesley Street West,\n\nFederal & Elliott \n\n,NaN,Auckland,09 366 1584,www.2degreesmobile.co.nz,No Obvious COVID Info,-36.850853,174.763871,Inside,0.2702


<h5>Superimpose Geofence Circle on the Map and Display</h5>

In [13]:
# Superimpose Geofence Circle
folium.Circle(
    radius=rad_m,
    location=[map_center_lat, map_center_lng],
    popup='The CBD Limits',
    color='#000000',
    fill=True,
    fill_color='#787A6F',
    fill_opacity=0.4
).add_to(akl_map)

# Display map
akl_map

<h5>Save DataFrame to Excel File</h5>

In [ ]:
akl_cbd_biz.to_excel(
    r'FULL FILE PATH\FILE.xlsx', index=False, header=True)  # Saved as "map_visualization_preprocessor.xlsx"

print("Saved to excel file")